<a href="https://colab.research.google.com/github/reachaki/compintelgroup16/blob/main/compintel16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate peft
import torch
import numpy as np
import random
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType


In [2]:
# For reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


In [3]:
dataset = load_dataset('emotion')
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [4]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6).to('cuda')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def tokenize_function(example):
    return tokenizer(
        example['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
small_train = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_val = tokenized_datasets["validation"]  # or .select(range(500)) for even faster evaluation


In [7]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    return {'eval_accuracy': acc}


In [8]:
def train_lora_model(lora_rank, lora_alpha, lora_dropout, batch_size=16, learning_rate=2e-5, num_epochs=3):
    from transformers import AutoModelForSequenceClassification
    model = AutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', num_labels=6).to('cuda')
    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=lora_alpha,
        target_modules=["q_lin", "v_lin"],
        lora_dropout=lora_dropout,
        bias="none",
        task_type=TaskType.SEQ_CLS,
        inference_mode=False
    )
    lora_model = get_peft_model(model, lora_config)
    lora_model = lora_model.to('cuda')
    training_args = TrainingArguments(
        output_dir='./results',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        logging_dir='./logs',
        report_to=[]
    )
    trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics
)

    trainer.train()
    eval_results = trainer.evaluate()
    accuracy = eval_results.get("eval_accuracy", None)
    print(f"Validation accuracy: {accuracy}")
    return accuracy


In [9]:
train_lora_model(lora_rank=2, lora_alpha=8, lora_dropout=0.15, num_epochs=5)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,1.515100
1000,1.165000
1500,0.968000
2000,0.847300
2500,0.762600
3000,0.727000
3500,0.696900
4000,0.661500
4500,0.654800
5000,0.653300


Validation accuracy: 0.774


0.774

In [ ]:
import random
import pandas as pd

# Define your search space
rank_choices = [2, 4, 8, 16]
alpha_choices = [8, 16, 32, 64]
dropout_choices = [0.05, 0.1, 0.15, 0.2]

results = []

num_trials = 5  # increase later for real experiments
for trial in range(num_trials):
    lora_rank = random.choice(rank_choices)
    lora_alpha = random.choice(alpha_choices)
    lora_dropout = random.choice(dropout_choices)
    print(f"Trial {trial+1}")
    val_acc = train_lora_model(lora_rank, lora_alpha, lora_dropout, num_epochs=1)
    results.append({
        'trial': trial+1,
        'lora_rank': lora_rank,
        'lora_alpha': lora_alpha,
        'lora_dropout': lora_dropout,
        'val_accuracy': val_acc
    })

# Save results to a DataFrame for easier analysis
results_df = pd.DataFrame(results)
print(results_df)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trial 1


Step,Training Loss
500,1.542400
1000,1.322300


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validation accuracy: 0.5355
Trial 2


Step,Training Loss
